In [1]:
# Reading the content of PDF which is a resume
import PyPDF2 #importing the library
pdf_file_obj = open('bcg-2022-annual-sustainability-report-apr-2023.pdf', 'rb')# opening the pdf file in read binary format

In [2]:
pdf_reader = PyPDF2.PdfReader(pdf_file_obj) # creating pdf reader object

In [3]:
num_pages = len(pdf_reader.pages) # reading the total pages

In [4]:
detected_text=''
for pages in range(num_pages):
    page_obj = pdf_reader.pages[pages] # reading the particular page
    detected_text+=page_obj.extract_text()+'\n\n' # extracting the content of page and saving in the created string
    
pdf_file_obj.close() # clossing the file 
print(detected_text)

2022 Annual Sustainability Report
Expanding Our Reach, 
Enhancing Our Impact  
April 2023

Boston Consulting Group partners with leaders 
in business and society to tackle their most 
important challenges and capture their greatest 
opportunities. BCG was the pioneer in business 
strategy when it was founded in 1963. Today, 
we work closely with clients to embrace a 
transformational approach aimed at benefiting  
all stakeholders—empowering organizations to 
grow, build sustainable competitive advantage, 
and drive positive societal impact.
Our diverse, global teams bring deep industry and 
functional expertise and a range of perspectives 
that question the status quo and spark change. 
BCG delivers solutions through leading-edge 
management consulting, technology and design, 
and corporate and digital ventures. We work in a 
uniquely collaborative model across the firm and 
throughout all levels of the client organization, 
fueled by the goal of helping our clients thrive and 
enabli

In [5]:
len(detected_text) # length of total text in pdf

251848

In [6]:
# lets chat with this pdf
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv

client = AzureOpenAI(
  azure_endpoint = os.getenv("openai_base"), 
  api_key=os.getenv("api_key"),  
  api_version=os.getenv("api_version")
)

def fun(text,Persona=None):
  if Persona is None:
    message_text = [{"role":"system","content":"You are an AI assistant that helps people find information."},
                {"role":"user","content":text}
                ]
  else:
    message_text = [{"role":"system","content":Persona},
                {"role":"user","content":text}
                ]  
  completion = client.chat.completions.create(
  model="GPT4-Turbo", # model = "deployment_name"
  messages = message_text,
  temperature=0.8,
  max_tokens=4000,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None
)
  print("Completion Tokens:",completion.usage.completion_tokens)
  print("Input Tokens :",completion.usage.prompt_tokens)
  return completion.choices[0].message.content.strip()
  

In [7]:
prompt="Summarize the below pdf in 500 words.\n\n{pdf}"

In [8]:
pdf=fun(prompt.format(pdf=detected_text))

Completion Tokens: 78
Input Tokens : 54969


In [10]:
# THe above pdf contains copyright content so it is not answering any question.

## Using LangChain

LangChain is an innovative technology that functions as a bridge - linking large language models (LLMs) with practical applications like Python programming, PDFs, CSV files, or databases.

In [57]:
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.chat_models import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings  
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
import os

In [58]:
# Splitting the text into chunks using Langchain text splitters

In [59]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)  #creating a object to convert the given data into chunks
texts=text_splitter.create_documents([detected_text]) #splitting the data into chunks

In [63]:
texts

[Document(page_content='2022 Annual Sustainability Report\nExpanding Our Reach, \nEnhancing Our Impact  \nApril 2023'),
 Document(page_content='Boston Consulting Group partners with leaders \nin business and society to tackle their most \nimportant challenges and capture their greatest \nopportunities. BCG was the pioneer in business \nstrategy when it was founded in 1963. Today, \nwe work closely with clients to embrace a \ntransformational approach aimed at benefiting  \nall stakeholders—empowering organizations to \ngrow, build sustainable competitive advantage, \nand drive positive societal impact.\nOur diverse, global teams bring deep industry and \nfunctional expertise and a range of perspectives \nthat question the status quo and spark change. \nBCG delivers solutions through leading-edge \nmanagement consulting, technology and design, \nand corporate and digital ventures. We work in a \nuniquely collaborative model across the firm and \nthroughout all levels of the client organ

In [14]:
# creating a vector database using the above chunks.
directory='index_store'
vector_index=FAISS.from_documents(texts,AzureOpenAIEmbeddings(  
    api_key=os.getenv("AZURE_OPENAI_EMBEDDING_KEY"),  
    azure_endpoint=os.getenv("AZURE_OPENAI_EMBEDDING_ENDPOINT"),  
    api_version=os.getenv("AZURE_OPENAI_VERSION"),  # Use the correct API version  
    model="text-embedding-ada-002",
    azure_deployment="Embedding"
)  )


In [15]:
vector_index.save_local(directory) 

The database index gets stored with the file name 'index_store'.

In [16]:
# Now Loading the database. Use this databse for retriever  and then create a chatbot

In [18]:
vector_index = FAISS.load_local('index_store',AzureOpenAIEmbeddings(  
    api_key=os.getenv("AZURE_OPENAI_EMBEDDING_KEY"),  
    azure_endpoint=os.getenv("AZURE_OPENAI_EMBEDDING_ENDPOINT"),  
    api_version=os.getenv("AZURE_OPENAI_VERSION"),  # Use the correct API version  
    model="text-embedding-ada-002",
    azure_deployment="Embedding"
) ,allow_dangerous_deserialization=True )

In [19]:
retriever = vector_index.as_retriever(search_type="similarity", search_kwargs={"k":6})   

In [21]:
qa_interface = RetrievalQA.from_chain_type(llm = AzureChatOpenAI(
    openai_api_type="azure",
    openai_api_key= os.getenv("AZURE_OPENAI_EMBEDDING_KEY"),
    openai_api_base=os.getenv("AZURE_OPENAI_EMBEDDING_ENDPOINT"),
    deployment_name="GPT4-Turbo",
    model="gpt-4",
    openai_api_version=os.getenv("AZURE_OPENAI_VERSION")
    
    ), chain_type="stuff", retriever=retriever, return_source_documents=True)

c:\Users\lenovo\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.azure_openai.AzureChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(
c:\Users\lenovo\anaconda3\Lib\site-packages\langchain_community\chat_models\azure_openai.py:167: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://aikillsseo.openai.azure.com/ to https://aikillsseo.openai.azure.com/openai.
  warnings.warn(
c:\Users\lenovo\anaconda3\Lib\site-packages\langchain_community\chat_models\azure_openai.py:174: UserWarning: As of openai>=1.0.0, if `

In [22]:
#Now its ready to chat with pdf
response = qa_interface("List measures taken to address diseases occuring in developing industries")

c:\Users\lenovo\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [26]:
print(response['result'])

In the context provided, several measures have been taken to address diseases occurring in developing countries, particularly through initiatives led by organizations like Pfizer and supported by consulting groups like the Boston Consulting Group (BCG). Here's a summary of the measures:

1. **An Accord for a Healthier World**: Pfizer announced this initiative aimed at providing access to its innovative portfolio of medicines and vaccines for people living in 45 lower-income countries. This program is designed to address health equity gaps and improve health outcomes for up to 1.2 billion people affected by deadly infectious diseases, certain cancers, and inflammatory diseases.

2. **Partnership Model Development**: To overcome the challenge of making the innovative portfolio available across 45 developing countries without individual go-to-market strategies, Pfizer sought assistance from BCG. Together, they developed a partnership model that mitigates risks, meets regulatory requiremen

So far, we've used the RetrievalQA chain, a LangChain type for pulling document pieces from a vector store and asking one question about them. But, sometimes we need to have a full conversation about a document, including referring to topics we've already talked about.
To make this possible, our system needs a memory or conversation history. Instead of the RetrievalQA chain, we'll use the ConversationalRetrievalChain.

In [27]:
conv_interface = ConversationalRetrievalChain.from_llm(AzureChatOpenAI(
    openai_api_type="azure",
    openai_api_key= os.getenv("AZURE_OPENAI_EMBEDDING_KEY"),
    openai_api_base=os.getenv("AZURE_OPENAI_EMBEDDING_ENDPOINT"),
    deployment_name="GPT4-Turbo",
    model="gpt-4",
    openai_api_version=os.getenv("AZURE_OPENAI_VERSION")
    
    ), retriever=retriever)

c:\Users\lenovo\anaconda3\Lib\site-packages\langchain_community\chat_models\azure_openai.py:167: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://aikillsseo.openai.azure.com/ to https://aikillsseo.openai.azure.com/openai.
  warnings.warn(
c:\Users\lenovo\anaconda3\Lib\site-packages\langchain_community\chat_models\azure_openai.py:174: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_name` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.warn(
c:\Users\lenovo\anaconda3\Lib\site-packages\langchain_community\chat_models\azure_openai.py:182: UserWarning: As of openai>=1.0.0, if `openai_api_base` (or alias `base_url`) is specified it is expected to be of the form https://example-resource.azure.openai.com/openai/d

In [54]:
chat_history = []
query = "in what context is Morocco mentioned in the report?"
result = conv_interface({"question": query, "chat_history": chat_history})
print(result["answer"])

Morocco is mentioned in the report in the context of a significant social protection reform, particularly focusing on expanding universal health care coverage to its population. The report outlines the challenges millions of Moroccans faced due to a widening gap in health equity, which became more apparent during the COVID-19 crisis. In response, the Moroccan government announced a comprehensive set of social reforms to be implemented over three years, from 2023 to 2025, aiming to improve health access and strengthen social protection programs.

The Boston Consulting Group (BCG) played a crucial role in assisting the Moroccan government with the reform, particularly in expanding access to universal health care. BCG's contributions included restructuring the reform agenda, focusing on increasing the quality of care, ensuring the financial sustainability of the programs, and considering the interdependencies between informational and regulatory systems. As a result of these efforts, by D

'chat_history' parameter is a list containing past conversation history. For the first message, this list will be empty.

'question' parameter is used to send our message.

In [55]:
# going forward by updating the chat_history 
chat_history.append((query, result["answer"]))

In [56]:
query = "give some statistics around this."
result = conv_interface({"question": query, "chat_history": chat_history})
print(result["answer"])

The report provides specific statistics regarding the expansion of universal health care coverage in Morocco as follows:

- More than 90% of Morocco’s population now have access to universal health care, up from 42% just months before the implementation of the reforms.
- This rapid increase in health care coverage is one of the most rapid in modern history.
- Around 10 million of Morocco’s most vulnerable citizens were integrated into the universal health care scheme within a matter of months.
